In [73]:
import jieba
import re
import jieba.posseg as pseg


# Demonstration of LDA and NMF for text analysis

In [75]:
def load_weicheng(filename='story.txt'):
    lines = open(filename).readlines()
    books = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('第') and line.endswith('章'):
            books.append(line)
        else:
            books[-1] += line
    for i in range(len(books)):
        books[i] = ' '.join(jieba.cut(books[i]))
    return books

def load_honglou(filename='honglou.txt'):
    lines = open(filename).readlines()
    book = [[]]
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if '书香屋' in line:
            book[-1].pop()
            book.append([])
        else:
            book[-1].append(line)
    return [' '.join(jieba.cut(''.join(chp))) for chp in book]

def load_xiyou(filename='xiyouji_wuchengen.txt'):
    
    def _is_sep(line):
        pat = re.compile(u'第.{1,3}回')
        if pat.search(line):
            return True
        return False
    
    lines = open(filename).readlines()
    book = [[]]
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if _is_sep(line):
            book.append([])
        else:
            book[-1].append(line)
    print(len(book))
    book = [''.join(chp) for chp in book if len(chp)]
    book_noun = []
    for chp in book:
        words = pseg.cut(chp)
        #book_noun.append( ' '.join([word for word, flag in words if flag[0] == 'n']))
        book_noun.append( ' '.join(jieba.cut(chp)))
    return book_noun

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx) + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]) )


documents = load_xiyou()
print()
no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=20, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 20
display_topics(nmf, tfidf_feature_names, no_top_words)
print('\nLDA')
display_topics(lda, tf_feature_names, no_top_words)

122



/users/sulixin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:行者 八戒 师父 妖精 一个 沙僧 唐僧 我们 怎么 那里 大圣 三藏 和尚 那怪 呆子 不知 甚么 两个 小妖 大王
Topic 1:意思 故事 后来 佛教 形容 称为 古代 一种 这里 比喻 这是 又称 所以 梵语 因为 道教 典故 人们 唐代 之一
Topic 2:国王 公主 驸马 陛下 八戒 关文 寡人 驿丞 长老 倒换 行者 国师 神僧 多官 娘娘 贫僧 文武 和尚 万岁 唐僧
Topic 3:三藏 长老 老者 师父 贫僧 圣僧 徒弟 十八 驿丞 和尚 袈裟 员外 老爷 女子 老母 不敢 大唐 师徒 我们 取经
Topic 4:大圣 天王 玉帝 李天王 太子 哪吒 真君 金星 妖猴 如来 猴王 蟠桃 齐天大圣 仙女 老君 二郎 四大 下界 天宫 圈子
Topic 5:祖师 悟空 猴王 美猴王 大王 龙王 花果山 神仙 水帘洞 乃是 大众 弟子 魔王 一个 甚么 金星 孙悟空 天地 兵器 观看
Topic 6:菩萨 如来 木叉 袈裟 取经 南海 莲台 观音 一个 弟子 大仙 锡杖 怪物 大众 王道 金刚 诸天 南无 宝杖 花果山
Topic 7:太宗 唐王 陛下 魏征 南无 判官 玄奘 金银 袈裟 阎王 法师 谢恩 御弟 文武 天下 锡杖 多官 光蕊 夫妻 传旨
Topic 8:道教 传说 先生 天尊 天师 道士 神仙 又称 婆婆 真人 这里 地方 八卦 认为 说法 古时 元帅 之一 天神 西游记
Topic 9:罗刹 扇子 牛王 大圣 芭蕉扇 土地 女子 芭蕉 牛魔王 猢狲 老者 魔王 公主 宝贝 变作 夫人 王道 宝剑 一只 金刚

LDA
Topic 0:三藏 师父 行者 形容 大圣 这里 一个 八戒 意思 后来 不知 唐僧 那里 不是 所以 古代 小龙 土地 故事 甚么
Topic 1:行者 八戒 师父 三藏 一个 大圣 沙僧 妖精 那里 怎么 唐僧 我们 不知 不是 菩萨 和尚 呆子 两个 只见 三个
Topic 2:意思 这里 佛教 道教 形容 后来 古代 一种 传说 称为 故事 比喻 神仙 所以 一个 这是 就是 地方 又称 认为
Topic 3:行者 八戒 师父 三藏 一个 唐僧 沙僧 怎么 那里 我们 大圣 和尚 妖精 不知 两个 菩萨 甚么 长老 不是 国王
Topic 4:菩萨 太宗 御弟 南无 女王 袈裟 玄奘 三藏 